# ID 5059 Coursework 1
John Belcher-Heath (jbh6)

# Introduction

The task is to predict the price of a car from a subset of attributes from the Kaggle dataset.

I will complete the task following the ML checklist in the book, Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow. which is:

1. Frame the problem
2. Get the data
3. Explore the data
4. Prepare the data
5. Explore models
6. Fine-tune models
7. Present solution
8. Launch/maintain

However, for the scope of this project 8. will not be needed and solutions will be presented in an external pdf report.

# 1. Frame the problem

We want to predict the price of a car (continuos) using a small selection of attributes available to us. This makes the problem a regression problem.

Since this is a regression problem the standard performance measure of Root Mean Square Error (referred to as RMSE from now on) will be used:

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^n(y_i - \hat{y}_i)^2}
$$

For this measure we are looking for low RMSE. This will mean small residuals and the model is a good fit for the data.

# 2. Get the data

In this section a random slection of entries from one of the large datasets will be obtained and read into a pandas.dataframe to explore. A random selection of the large dataset will be explored since all we are doing is getting to know the data. Having a large amount of data to explore will be time consuming, but having too small (and non random sample) will mean our observations may not be valid. Taking a random sample of a large dataset should give a relatively good representation of the overall dataset, whilst minimising the amount of data requiring to be manipulated.

Note when it comes to applying the model I will include a check of the data to make sure our observations on the smaller dataset still hold.

In [ ]:
import sys
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn | grep -v 'already satisfied'

# Import libraries
import pandas as pd
import numpy as np
import sklearn
import os
import glob
from pathlib import Path
import math

In [ ]:
# folder_path: str = "/cs/studres/ID5059/Coursework/Coursework-1/data/2_medium" # uni
folder_path : str = r"/home/johnbh/personal_git/ID5059_coursework_1/data/3_large" # Desktop

if not os.path.exists(folder_path):
    raise FileNotFoundError
os.chdir(folder_path)

file_names : list = [i for i in glob.glob("*.{}".format('csv'))]

    
def read_car_data(filepath : str) -> pd.DataFrame:
    """
    Reads a filepath and returns the dataframe
    :param filepath: The location of the file to read
    :return: returns the pandas dataframe
    """
    return pd.read_csv(filepath)#, index_col = "vin")

original_df: pd.DataFrame = read_car_data(file_names[0])

# Clear the maximum number of columns to be displayed, so that all will be visible.
pd.set_option('display.max_columns', None)
# check data looks roughly okay
original_df.head(5)

# 3. Explore the data

The data will now be inspected to explore what attributes are available to using the info output. Attributes with large proportion of NAs can start to be identified as well.

In [ ]:
# original_df: pd.DataFrame = original_df.reset_index(drop=True) # Reindex to make elements easier to quickly access
original_df.info()

In [ ]:
# Explore attributes
original_df.head(5)

Initial observations from head:

- A lot of measurements contain the units, making the non-numerical
- Descriptions contain lots of irrelevant information
- A few columns seem to represent the same information
- Some attributes appear to have lots of NaNs
- Multiple ID attributes which can all be dropped
- `major_options` is a list which will need parsing somehow
- `power` contains all the info of `horsepower`
- Lots of irrelevant metadata to drop

Let's split the data and assign the labels

In [ ]:
from sklearn.model_selection import train_test_split

split_train: float = 0.6# fraction of data to use to explore

train_set, test_set = train_test_split(original_df, test_size = split_train, random_state=314)

train_set_index, test_set_index = train_set.index, test_set.index

prices = original_df.price.copy() # Takes from test and training, useful later

df = train_set.copy() # copy so can recover if needs
sample_size: int = len(train_set)

### Start to inspect
Firstly, let's drop all attributes from above which have less than 50% non-null values, since including these may negatively effect our model if a majority of entries do not have this attribute. Using them in our model will mean the model is not very general.

In [ ]:
# Drop all attributes with less than 50% non-null values
df = df.drop(columns=df.keys()[df.count() / sample_size < 0.5])

### Data types correction
Some of the attributes appear to have been imported with different datatype, for example `zip code` as `object` not `int64`. This will be due to some integer attributes containing `NaNs`, and since the system has no interpretation for `NaNs` in `integer` types, they are taken as `object` data types instead. 

To further inspect this, all `object` data types are shown below.

In [ ]:
df.select_dtypes(include=object).info()

From manual inspection there are some attributes that need further inspection to check they have been given the correct type. The first 5 entries are shown below to help.

In [ ]:
pd.set_option('display.max_columns', None)
df.select_dtypes(include=object).head(5)

The only attribute that can be directly converted to an integer is the `dealer_zip`, this is unlikely to provide any additional information that the `lattitude` and `longitutde` will not already give so no need to convert.This is dropped from our dataset below. 

This inspection has shown that a lot of the measurements have had units included, so these attributes will need to be converted to numerical.

In [ ]:
# Drop dealer_zip
try:
    df = df.drop(columns='dealer_zip')
except KeyError:
    print("Column already dropped")

In [ ]:
def convert_measurement(s: str) -> float:
    """
    Converts the measuremnt with units to a numerical value
    :param s: string measurement
    :type s: str
    :return: the actual numerical value
    """
    if type(s) == str:
        s_split: list = s.split(" ")
        try:
            return float(s_split[0])
        # If cannot convert to dtype, ie NA then return NA
        except ValueError:
            return float('NaN')
    # If already converted to correct format, ie if function accidently run twice
    else:
        return s

cols_to_convert: list = ["back_legroom", "front_legroom", "fuel_tank_volume", "height", "length", 
                         "maximum_seating", "wheelbase", "width"]

In [ ]:
# Apply the function to get numerical data from the string measurements
df[cols_to_convert] = df[cols_to_convert].applymap(convert_measurement)
df[cols_to_convert] = df[cols_to_convert].astype(np.float64)

It is important to note that the attributes `power` and `torque` contain numerical data, but they cannot be simply convert at this point but will be saved for later.

Next, let's drop all the irrelevant meta data which won't be helpful with our model and will instead just increase the complexity which could lead to overfitting. For example the `description`, `interior color`, `exterior color`, `vin` etc.

In [ ]:
df = df.drop(columns=['description', 'interior_color', 'exterior_color', 
                      'main_picture_url', 'model_name', 'sp_name', 'transmission_display',
                      'trim_name', 'trimId', 'vin', 'sp_id', 'listing_id'])

### Fixing duplicates part 1

It is easy to see that `engine_cylinders` and `engine_type` appear to be duplicate. Similarly so do `wheel_system` and `wheel_system_display`, as well as `make_name` and `franchise_make`.

Before dropping one of each of these, the data will be further inspected to make sure that there's no discrepancy between the two in the wider data set (i.e. not just in the head).

In [ ]:
df_engine = df[['engine_cylinders', 'engine_type']]
df_engine[np.logical_xor(df_engine.engine_cylinders.isna(), df_engine.engine_type.isna())].count()

So above tells us that all entries with attributes are identical in being either NA or not, so dropping one of these attributes means no information is lost.

In [ ]:
df = df.drop(columns='engine_cylinders')

For the `wheel_system` and `wheel_system_display`:

In [ ]:
df_wheel = df[['wheel_system', 'wheel_system_display']]
df_wheel[np.logical_xor(df_wheel.wheel_system.isna(), df_wheel.wheel_system_display.isna())].count()

The above implies that both attributes provide the same information for the cars. Hence deciding which to drop is irrelevant. I will choose to drop the `wheel_system_display` since wheel system has a nice short appriviation.

In [ ]:
df = df.drop(columns='wheel_system_display')

Finally for make.

In [ ]:
df_make = df[['make_name', 'franchise_make']]
df_make[np.logical_xor(df_make.make_name.isna(), df_make.franchise_make.isna())].count()

From this we can see that the `make_name` has more information than the `franchise_make`, hence the `franchise_make` is dropped.

In [ ]:
df = df.drop(columns='franchise_make')

### Fixing duplicates part 2

For part 2, these duplicates data may need to be extracted then compared, before just dropping attributes.

Let's inspect the engine data:

In [ ]:
df[np.logical_xor(df.engine_displacement.isna(), df.horsepower.isna())]

So, luckily `horsepower` and `power` do give the same information so one can be dropped arbitrarily. As horsepower is already numerical, `power` will be dropped.

In [ ]:
df = df.drop('power', axis=1)

There is also another useful attribute of RPM which could help to distinguish between performance cars with large horsepower and 4x4 with the same, but there may be too many NAs for this attribute to use this metric, let's see.

In [ ]:
df.horsepower.count() / sample_size

So from above we can see that only around 5% have no `horsepower` attribute. For these remaining entries we will consider how many have engine size attributes.

In [ ]:
len(df[(df['horsepower'].isna() & df['engine_type'].isna())]) / sample_size

In [ ]:
################ REDO

Now there is only a small amount of cars with neither `horsepower` or `engine_type` attribute. All these entries will simply take the overall median for `horsepower`.

The `horsepower` for all cars will be assigned using the following:

- if the car has `horspower` asigned pass
- elif the car has `engine_type` assign median for that type
- else assign the overall median for `horsepower`

Let's do it:

In [ ]:
df[['horsepower']] = df[['horsepower', 'engine_type']].groupby('engine_type').transform(lambda x: x.fillna(x.median()))

**Note: this is an imputation step so we don't want to use horsepower as a way to fill other NaNs as this could lead to too muhc creation of data and introduce bias**

Let's examine the improvements

In [ ]:
df.horsepower.count() / sample_size

Now for the final step of assigning the last na just the average of all the horsepowers:

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df[['horsepower']])

df[['horsepower']] = imputer.transform(df[['horsepower']])

Let's see the results

In [ ]:
df.horsepower.count() / sample_size, df.horsepower.hist()

Everything looks all good!

In [ ]:
####################################################################

### Object type attributes
Now we have removed some of the duplicates and corrected some of the data type issues the `object` type attributes will be properly explored now.

In [ ]:
df.select_dtypes(include=object).head(5)

First let's see if any of the attributes have any blaring issues with NAs.

In [ ]:
df.select_dtypes(include=object).count() / sample_size

Clerly some of the attributes are not suitable to use since they have a low number of entries. Any object attributes with less than 80% entries are removed.

In [ ]:
df = df.drop(columns=df.select_dtypes(include=object).loc[:, df.select_dtypes(include=object).count() / sample_size < 0.8].keys())

This leaves:

In [ ]:
df.select_dtypes(include=object).head(5)

Since we have `daysonmarket` attribute the `listed_date` can be dropped. Additionally, `city` can assumed to have minimal effect since most cities can be assumed to have a diverse range of individuals with varying wealth and cars.

In [ ]:
df = df.drop(columns=['city', 'listed_date'])

`torque` could be useful but there is too few entries (see below) for it and it is not recorded elsewhere (like `horsepower` recorded in `power` and `engine_size`). Hence I will not use this attribute for my model

In [ ]:
df.torque.count() / sample_size

In [ ]:
df = df.drop(columns='torque')

For major options, since there is so much variabilty from visual inspection of naming of products, the number of major of features will be used instead. The actual usefulness of this will be explored later.

In [ ]:
df.major_options = df.major_options.apply(lambda x: len(x.split(",")) if type(x) == str else "NaN").astype(np.float64)

For the remaining attributes, these will be used as categorical attributes in the model.

In [ ]:
chosen_categorical_attributes : list = df.select_dtypes(include=object).keys().to_list()

### Explore the bool values
Next let's explore the boolean attributes available

In [ ]:
df.select_dtypes(include=bool).head(5)

Both these could be useful, let's see if there's any issues with NaNs

In [ ]:
df.select_dtypes(include=bool).count() / sample_size

Fantastic! Both these attriubtes have no NaNs so can be used straight away.

In [ ]:
chosen_boolean_attributes : list = ['franchise_dealer', 'is_new']

### Exploring the numerical attributes

Now the qualitative attributes have been dealt with it's time for the quantiative attributes.

Let's explore all the numerical attributes with an actual numerical meaning(index or listing_id have no meaning numerically). Attributes with no numerical meaning our dropped below.

In [ ]:
# Quick inspection to see which numerical but non-relevant attributes need to be dropped
df.select_dtypes(include=[np.int64, np.float64])

In [ ]:
import matplotlib.pyplot as mpl
%matplotlib inline

df_numerical = df.select_dtypes(include=[np.int64, np.float64])
df_numerical.hist(figsize=(16,20), bins=30)
mpl.show()

Observations:
- Both Fuel economy attributes appear to be normally distributed with a slight skew
- Majority of cars do not stay on the market for a long duration, mostly less than a couple of months. Some may be above a large amount so these may need to be removed to not skew data.
- Engine displacement doesn't appear to have any obvious standard distribution
- Horsepower appears to have a normal distribution around 200hp with a standard deviation of around 50hp
- Lattitude is as expected all grouped together around 39 to 44 
- longitutde is split into two peaks, most likely corresponding to central US and alaska
- Milegae of most cars is grouped mostly around 0 and fewer cars with higher mileage, as would be expected
- owner count has a modal of 1, again as to be expected
- Most cars prices are group around the same order of magnitutde. Howeever some extremes are seen. A logarithmic transformation may need to be considered later.
- Seller ratings appear to be skew negatively towards the higher end
- Majority of cars are from the last 15 years
- Modal max seats is 5

It is clear as well that some of the bins are very sparse so will need coarser bins with labels for our model later to make sure our training set and test set have similar distributions.

In [ ]:
df.select_dtypes(include=[np.float64, np.int64]).count() / sample_size

Firstly it is clear to see there is no issue with NAs in the attributes: `daysonmarket`, `lattitude`, `longitude`, `price`, `savings_amount` and `year` (as well as `horsepower` after the fix above). Using contextual knowledge all these attributes (excl `price` as this is being compared to) will likely be useful in predicting the `price` attribute so will be used. 

Looking at the list of other attributes available with a low number of non-nulls. The additional attributes I believe may effect the `price` and want to explore more are:

- `city_fuel_economy` and `highway_fuel_economy` - useful metric of car performance, more powerful and expensive cars likely to have lower fuel efficiency
- `fuel_tank_volume` - bigger more expensive cars likely to have a large fuel tank, hence useful metric
- `engine_displacement` and `horsepower` (and `power` which will be used to get na values) - all similar/the same metrics for how powerful a car is
- `major_options` - more expensive cars tend to have more options
- `mileage` - more miles done the less it is valued generally
- `seller_rating` - If a seller has a better rating people may pay more than if they were to go to a seller with a poor rating.
- `length` and `width` - A measure of the size of the car. Large cars tend to be more expensive. E.g. sports cars are very wide generally.

I have chosen not to include `owner_count` since there are too few entries for this attribute. Additonally `maximum_seating` is also excluded, although small number of non-nulls, this is since cars seating will have little effect on price. Think two seat sports cars and a smart car, or a 5 seat corsa and a 5 seat golf.

To explore these options there is some transformation required to remove any skew by the extreme values, also to reduce the complexity of the model.

### Attribute transformation
From the graphs above some attributes we have chosen to explore further need transforming so that the distribution of the training set and test set are similar. To do this the function below will be used.

**The function will be demonstrated but not applied to the data yet since there are NAs that need filling**

In [ ]:
### NEED TO TWEAK
def transform_bins(pds: pd.Series, bins, min_val = None, max_val = None) -> pd.Series:
    """
    Function to transform a continuous series with sparse data to a categorical attribute with full bins.
    The absolute max is always 0 and inf to make sure all data is captured.
    :param pds: original cts data
    :param bins: number of bins in resultant series (note this is how many will be attempted to be created)
    :min_val: starting value for main section of the bins
    :max_val: ending value for main section of the bins
    :return: transformed series
    """
    bins -=1
    if min_val is not None and max_val is not None: 
        cuts: list = np.append(np.linspace(min_val, max_val, bins), np.array([np.inf])).tolist()
        cuts.insert(0,0)
    else:
        cuts: list = np.append(np.linspace(pds.quantile(0.025), pds.quantile(0.975), bins), np.array([np.inf])).tolist()
        cuts.insert(0, 0)
        
    # Drop any duplicates, ie if 0 included twice
    cuts = list(dict.fromkeys(cuts))
    labels: list = [str(i) for i in range(len(cuts)-1)]
    # include_lowest needed to make sure if values are 0 they're still given a label
    return pd.cut(pds, bins=cuts, labels=labels, include_lowest=True).astype(np.float64)

The attributes needing to be transformed are:

In [ ]:
transform_attributes: list = ["city_fuel_economy", "highway_fuel_economy","daysonmarket", "fuel_tank_volume", 
                              "mileage", "savings_amount", "year", "major_options"]

The function will be applied in a uniform way with 30 bins for each first, these will then be inspected to see if more detailed transformation may be required

In [ ]:
transformed_attr: pd.DataFrame = df[transform_attributes].apply(lambda x: transform_bins(x, bins=30))
transformed_attr.hist(figsize=(16,16))
mpl.show()

These distributions look much better than before. However there may be a slight issue with `savings_amount` and `city_fuel_economy`. For this one different min, max and bins need to be used. Using contextual knowledge the following conversions are used.

In [ ]:
transformed_attr[['city_fuel_economy']] = df[['city_fuel_economy']].apply(lambda x: transform_bins(x, bins=5, min_val=18, max_val=28))

In [ ]:
transformed_attr[['savings_amount']] = df[['savings_amount']].apply(lambda x: transform_bins(x, bins=5, min_val=100, max_val=3000))

In [ ]:
transformed_attr.hist(figsize=(16,16))

These look much better than before.

After all the exploratory analysis a list of attributes which are hopefully correlated to the `price` attribute have been identified. But before preperation let's take a look at the correlation between the numerical attributes and the `price` to maybe eliminate some attributes, reducing the complexity.

### Explore correlations

In [ ]:
chosen_numerical_attributes : list = ['daysonmarket', 'latitude', 'longitude', 'price', 'savings_amount', 'year', 'horsepower', 'city_fuel_economy', 
'highway_fuel_economy', 'fuel_tank_volume', 'engine_displacement', 'major_options', 'mileage', 'seller_rating',
'length', 'width']

In [ ]:
df_numerical = df.select_dtypes(include=[np.float64, np.int64])[['daysonmarket', 'latitude', 'longitude', 'price', 'savings_amount', 'year', 'horsepower', 'city_fuel_economy', 
'highway_fuel_economy', 'fuel_tank_volume', 'engine_displacement', 'major_options', 'mileage', 'seller_rating',
'length', 'width', 'wheelbase']]
# abs taken as don't care if posotive or negative effect
corr_series = abs(df_numerical.drop("price", axis=1).apply(lambda x: x.corr(df_numerical.price)))
corr_series.sort_values()

Clearly some the attributes left don't have much of a correlation
Now let's choose all attributes with a correlation of more than 0.25 and use some of our contextual knowledge to inspect.

In [ ]:
corr_series[corr_series > 0.25]

All these attributes seem to make logical sense. One attribute that could be removed is one of `wheelbase` or `length` since they represent different ways to measure the length of a car. Since wheelbase has the higher correlation, `length` will be dropped. Let's inspect the above attributes in more detail. Any attributes we had intially chosen but have are not included above will be dropped.

In [ ]:
chosen_numerical_attributes = corr_series[corr_series > 0.25].keys().tolist()
chosen_numerical_attributes.remove('length')
chosen_numerical_attributes.append('price')

In [ ]:
df[chosen_numerical_attributes].info()

In [ ]:
pd.plotting.scatter_matrix(df[chosen_numerical_attributes], figsize=(15,15))
mpl.show()

Inspecting the `price` row (or column), `horsepower` and `mileage` have the stongest correlation as to be expected. 
`wheelbase` and `width` appear to have similar correlation to price, which is to be expected by them being a measurement of size.

## 4. Prepare data

Now the intial exploration and some of the transformation needed have been identified. The data can start to be prepared.

!!!!!!!!!!!!!!!!!!!!!!!!!!
Seperate the labels from the data:

In [ ]:
#prices = original_df[["price"]].copy() # Takes from test and training, useful later
#prices.head(5)

In [ ]:
# Drop price from dataframe
#df = df.drop(columns='price')
#df.head(5)

In [ ]:
#########################################

In [ ]:
chosen_attributes : list = chosen_numerical_attributes + chosen_categorical_attributes + chosen_boolean_attributes
chosen_attributes.remove('price')
df = df[chosen_attributes].copy()

Check out any remaining data values which are missing:

In [ ]:
df.count() / sample_size

### Imputation
Let's fix the null values.

Now we want to start thinking about making our pipeline, let's make some custom Imputers to act on our data. Note the imputer created will be used in place of the process for `horsepower` for ease.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class WithinGroupImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, group_var):
        self.group_var = group_var
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        X_ = X.copy()
        for col in X_.drop(self.group_var, axis=1).columns:
            if X_[col].dtypes == 'float64':
                # For float types impute with median of group or overall if not available
                X_.loc[(X[col].isna()) & X_[self.group_var].notna(), col] = X_[self.group_var].map(X_.groupby(self.group_var)[col].median())
                X_[col] = X_[col].fillna(X_[col].median())
            if X_[col].dtypes == 'object':
                # For object types impute with mode of group or overall if not available
                X_.loc[(X[col].isna()) & X_[self.group_var].notna(), col] = X_[self.group_var].map(X_.groupby(self.group_var)[col].agg(pd.Series.mode))
                overall_mode = str(X_[self.group_var].mode().tolist()[0])
                # Library has issues with object atttributes and NaN so need to replace them with np.NaN explciitly
                X_.loc[:, col] = X_[col].fillna(np.nan).replace(np.nan, overall_mode)
        return X_

One important thing for imputations I have done is to make sure that any imputations are done from the raw data only, and not from other imputations. This reduces any bias introduced. So for example if the mode of groups is assigned to an attribute, the groups the cars belong to are not found by imputation themselves.

Similar to what happened for `horsepower `, `wheelbase` and `width` can assumed to be similar measures of a cars size and hence the `body_type` can be used as a proxy. This is chosen for the high number of non-null values. For any which don't have the `body_type` attribute the overall average will be used. For this imputation we will use the cust `WithinGroupImputer`.

In [ ]:
body_group_imp = WithinGroupImputer(group_var='body_type')
df[['wheelbase']] = body_group_imp.fit_transform(df[['wheelbase', 'body_type']])[['wheelbase']]
df[['width']] = body_group_imp.fit_transform(df[['width', 'body_type']])[['width']]

In [ ]:
# Check everything worked
df[['wheelbase', 'width']].count() / sample_size

For `mileage`, `year` will be used as a proxy, with the median from that year assigned, median is used as should be a nice symmetrical distribution for each year. An example to support this is shown below.

In [ ]:
df[df.year == 2015].mileage.hist()

In [ ]:
year_group_imp = WithinGroupImputer(group_var='year')
df[['mileage']] = year_group_imp.fit_transform(df[['mileage', 'year']])[['mileage']]

In [ ]:
# Check everything worked
df.mileage.count() / sample_size

For `major_options` the median will be taken, since the number of different possible options is low and a central discrete metric is desried. For this a `SimpleImputer` is used.

In [ ]:
median_imp = SimpleImputer(missing_values=np.nan, strategy='median')
df[['major_options']] = median_imp.fit_transform(df[['major_options']])

For `wheel_system` the `body_type` will be used as a proxy, and the mode for the given body type will be taken as the value. Note this imputation is done before the imputation of `body_type` to reduce inducing bias.

In [ ]:
df[['wheel_system']] = body_group_imp.fit_transform(df[['wheel_system', 'body_type']])[['wheel_system']]

For `fuel_tank_volume` this is likely proportional to the type of car, so the median of the `fuel_tank_volume` for the `body_type` can be used as proxy. Think about hatchbacks vs trucks.

In [ ]:
df[['fuel_tank_volume']] = body_group_imp.fit_transform(df[['fuel_tank_volume', 'body_type']])[['fuel_tank_volume']]

Similarly for `transmission` as few entries missing, the mode will be used.

In [ ]:
mode_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[['transmission']] = mode_imp.fit_transform(df[['transmission']])

For `fuel_type` let's examine the attribute in more detial

In [ ]:
df.fuel_type.value_counts()

Clearly an overwhelming majority is Gasoline, so let's fill all NaNs with Gasoline.

In [ ]:
df['fuel_type'] = df['fuel_type'].fillna('Gasoline')

For `engine_displacement` the attribute is given the median for an entry with a given `engine_type`, then filled with the overall median if no `engine_type` is present.

In [ ]:
engine_group_imp = WithinGroupImputer(group_var='engine_type')
df[['engine_displacement']] = engine_group_imp.fit_transform(df[['engine_displacement', 'engine_type']])[['engine_displacement']]

In [ ]:
########################## COME BACK TO THIS #####################################

Now let's deal with `engine_type`. Let's take a closer look at the options available a bit closer:

In [ ]:
df.engine_type.value_counts()

Inital thoughts are there a lot of different categories so this may be a difficult attribute to fix. Let's explore the the relation between `make_name` and `engine_type`.

In [ ]:
groups = df.groupby('make_name')
pd.set_option("display.max_rows", None)
groups.engine_type.value_counts()

In [ ]:
# Add max rows back
pd.set_option("display.max_rows", 10)

From the above with visual inspection if the mode for a given `make_name` is taken as the `engine_type` this should give a good imputation. Logically most manufactures will mass produce one or similar engine types for all vehicles to reduce production costs. Although there is some outliers in this since there is only a few to fill, this simple method is chosen.

In [ ]:
make_name_imp = WithinGroupImputer(group_var='make_name')
df[['engine_type']] = make_name_imp.fit_transform(df[['engine_type', 'make_name']])[['engine_type']]

Let's check this worked okay

In [ ]:
df.engine_type.value_counts()

So some entries have been given an empty array instead of a scalar. This may be due to some brands having only one car which have NaNs for the `engine_type`. Let's take a look deeper:

In [ ]:
df[['engine_type', 'make_name']].groupby('make_name').agg(pd.Series.mode)

That appears to be the case. 

So with these, instead of making the transformer any more complicated, we will just apply an extra step to the output

In [ ]:
mask = df['engine_type'].str.len() == 0
overall_mode = df['engine_type'].mode()[0]
df.loc[mask, 'engine_type'] = overall_mode

For `body_type` since there is very few entries missing, the mode for the attribute would make sense for this catagorical attribute.

In [ ]:
df[['body_type']] = mode_imp.fit_transform(df[['body_type']])

Let's check all the data is looking good.

In [ ]:
df.keys()[df.isna().any()]

Fantastic, all is good!

Now all NaNs have been dealt with let's see if the transformations discussed earlier still need applying. The only attributes from before that we transformed left are:
- `fuel_tank_volume` 
- `mileage`
- `year`

In [ ]:
df.hist(figsize=(16,16))

`year` and `mileage` appear to be the only ones that still need transforming from before. Let's do it same as before:

In [ ]:
transform_attributes: list = ['mileage', 'year', 'major_options']
df[transform_attributes] = df[transform_attributes].apply(lambda x: transform_bins(x, bins=30))
df[transform_attributes].hist()

These look much better than before.

The next step is to encode the categorical data to be meaning full.

So for all these attribute there is no intrinsic ranking. Although `engine_type` could be, there is no factual way of saying which is 'better'. So, one-hot-encoding will be used,

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

categorical_data : list = df.select_dtypes(include='object').keys().tolist()
df_categorical_encoded = one_hot_encoder.fit_transform(df[categorical_data].astype('str'))

Let's do the same for the boolean attributes now.

In [ ]:
bool_data: list = df.select_dtypes(bool).keys().tolist()
df_bool_encoded = one_hot_encoder.fit_transform(df[bool_data])

Final step for our data is to scale the numerical features so the algorithgms work well. We'll go with the standardisation scaling since most of the data does not have massively large tails. The only ones that may pose an issue are mileage and year, since they do have longer tails than the others.

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_data: list = df.select_dtypes(include=[np.int64, np.float64]).keys().tolist()

scaler = StandardScaler()
df_numerical_encoded = pd.DataFrame(scaler.fit_transform(df[numerical_data]), columns = numerical_data)
df_numerical_encoded.head(5)

Let's check they look okay:

In [ ]:
df_numerical_encoded.hist(figsize=(16,16))
mpl.show()

They all look good. Let's carry on.

### Transformation pipelines
Now we have the data preproccessed in the desired form, let's turn this into a pipeline.

Firstly, let's get the names of all the attributes we need from the dataset. And print the head of the orignial `train_set` so the index of each attribute can be seen. This is need for some section of the pipeline as the data is passed as numpy arrays, so all index and attributes need to match correctly.*

*We don't worry about this before as the columns are assigned directly one at a time so numpy form is no issue

In [ ]:
attributes : list = df.keys().to_list()
print(attributes)
train_set[attributes].head(5)

Next let's inspect the attributes

In [ ]:
numerical_data, categorical_data, bool_data

In [ ]:
all_attributes = numerical_data + categorical_data + bool_data
all_attributes

For numerical data we need to remeber that `year`, `major_options` and `mileage` are in indicies 0, 4 and 5 respectively.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

class ExctractAttributesTransform(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.cols_to_convert = ['fuel_tank_volume', 'wheelbase', 'width']
        self.cols_to_summarise = 'major_options'
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_ = X.copy()
        X_[self.cols_to_convert] = X_[self.cols_to_convert].applymap(convert_measurement).astype(np.float64)
        X_[self.cols_to_summarise] = X_[self.cols_to_summarise].apply(lambda x: len(x.split(",")) if type(x) == str else "NaN").astype(np.float64)
        return X_

class WithinGroupImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, group_var, attr):
        self.group_var = group_var
        self.attr = attr
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        X_ = X.copy()
        for col in self.attr:
            if X_[col].dtypes == 'float64':
                # For float types impute with median of group or overall if not available
                X_.loc[(X_[col].isna()) & X_[self.group_var].notna(), col] = X_[self.group_var].map(X_.groupby(self.group_var)[col].median())
                X_[col] = X_[col].fillna(X_[col].median())
            if X_[col].dtypes == 'object':
                # For object types impute with mode of group or overall if not available
                X_.loc[(X_[col].isna()) & X_[self.group_var].notna(), col] = X_[self.group_var].map(X_.groupby(self.group_var)[col].agg(pd.Series.mode))
                overall_mode = str(X_[self.group_var].mode().tolist()[0])
                # Library has issues with object atttributes and NaN so need to replace them with np.NaN explciitly
                X_.loc[:, col] = X_[col].fillna(np.nan).replace(np.nan, overall_mode)
        return X_
    
class columnDropperTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns=columns

    def fit(self, X, y=None):
        return self 
    
    def transform(self,X):
        return X.drop(self.columns,axis=1)

class dataFix(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_ = X.copy()
        mask = X_['engine_type'].str.len() == 0
        overall_mode = X_['engine_type'].mode()[0]
        X_.loc[mask, 'engine_type'] = overall_mode
        return X_.astype(str) # need to convert for encoder
    
class transformNumericalAttributes(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        # Indexes corresponding to year, mileage and major_options
        self.transform_attributes_index = [0, 4, 5]
        
    def fit(self, X, y=None):
        return self
            
    def transform(self, X):
        X_ = X.copy()
        df = pd.DataFrame(X_[:, self.transform_attributes_index]).apply(lambda x: transform_bins(x, bins=30))
        X_[:, self.transform_attributes_index] = df
        return X_
    
numerical_pipeline = Pipeline([
        ('extract_numericals', ExctractAttributesTransform()),
        ('impute_num_body_groupby', WithinGroupImputer('body_type', ['wheelbase', 'width', 'fuel_tank_volume'])),
        ('impute_num,_year_groupby', WithinGroupImputer('year',  ['mileage'])),
        ('impute_num_engine_groupby', WithinGroupImputer('engine_type', ['engine_displacement', 'horsepower'])),
        ('drop_grouping_attr', columnDropperTransformer(columns=['body_type', 'engine_type'])),
    
        # Fills any remaining numerical NaN with mdeian, should only apply tomajor_options
        # Note this step returns a numpy array so must use index from now on
        ('basic_impute', SimpleImputer(strategy='median')),
        ('transform_skewed', transformNumericalAttributes()),
        ('scaler', StandardScaler())
    ])

categorical_pipeline = Pipeline([
    ('impute_cat_body_groupby', WithinGroupImputer('body_type', ['wheel_system'])),
    ('impute_cat_make_groupby', WithinGroupImputer('make_name', ['engine_type'])),
    ('fixes', dataFix()),
    # Basic mode for transmission, body_type and fuel_type
    ('basic_mode_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('encoding', OneHotEncoder())
])

bool_pipeline = Pipeline([
    ('encoding', OneHotEncoder())
])

full_pipeline = ColumnTransformer([
    ("numerical", numerical_pipeline, (numerical_data + ['body_type', 'engine_type'])),
    ("categorical", categorical_pipeline, categorical_data),
    ("bool", bool_pipeline, bool_data)
])

prepared_data = full_pipeline.fit_transform(train_set[all_attributes])

Let's have a quick look at the data

In [ ]:
pd.DataFrame(prepared_data.toarray()).head(10)

Clearly there is a lot of attribute, this may mean the model takes a while to fit, it could be worth reducing attributes in the future to reduce training time.

!!!!!!!!!!!!!!!!!!!!!!!!
Since we are runnning it on a home computer, I will purely consider the first 7 attributes and last 2 which correspond to the numeric attributes and the two boolean attributes.
!!!!!!!!!!!!!!!!!!!!!!!

### Encoding the data
Now the data set is ready to be encoded correctly for the models. !!!!!!!!!!!!!!!!!!!!!!!!!!!!

## 5. Exploring models 
### Linear regression model
Since `price` is a numerical attribute a linear regression model makes sense. Let's try this model first.

In [ ]:
df_prepared_reduced = prepared_data[:,[0,1,2,3,4,5,6,7,-1,-2]]

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(df_prepared_reduced, prices[train_set_index])

Let's sample some random test data from the `test_set`

In [ ]:
test_data = test_set.sample(30)
pd.DataFrame(test_data)

In [ ]:
test_labels = prices[test_data.index]
pd.DataFrame(test_labels)

In [ ]:
test_data_prepared = full_pipeline.fit_transform(test_data)[:,[0,1,2,3,4,5,6,7,-1,-2]]
pd.DataFrame(test_data_prepared.toarray())

In [ ]:
test_predictions = linear_regression.predict(test_data_prepared).round()
test_predictions

In [ ]:
test_labels

In [ ]:
from sklearn.metrics import mean_squared_error

linear_regression_price_predictions = linear_regression.predict(test_data_prepared)
linear_regression_mse = mean_squared_error(test_labels, linear_regression_price_predictions)
linear_regression_rmse = np.sqrt(linear_regression_mse)
np.round(linear_regression_rmse)

In [ ]:
np.round(linear_regression_rmse / test_labels.median(), 2)

### Decision tree model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_regressor = DecisionTreeRegressor(random_state=314)
tree_regressor.fit(df_prepared_reduced, prices[train_set_index])

In [ ]:
test_data

In [ ]:
from sklearn.model_selection import cross_val_score

K = 10

tree_regressor_scores = cross_val_score(tree_regressor, df_prepared_reduced, prices[train_set_index],
                         scoring="neg_mean_squared_error", cv=K)

In [ ]:
tree_regressor_rmse_scores = np.sqrt(-tree_regressor_scores)

In [ ]:
def display_scores(scores):
    print("Scores:", np.round(scores))
    print("Mean:", np.round(scores.mean()))
    print("Standard deviation:", np.round(scores.std()))

display_scores(tree_regressor_rmse_scores)

In [ ]:
linear_regression_scores = cross_val_score(linear_regression, df_prepared_reduced, prices[train_set_index],
                                           scoring="neg_mean_squared_error", cv=K)
linear_regression_rmse_scores = np.sqrt(-linear_regression_scores)
display_scores(linear_regression_rmse_scores)

### Random forest regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


forest_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=14)
forest_regressor.fit(df_prepared_reduced, prices[train_set_index])

forest_regressor_prices_predictions = forest_regressor.predict(df_prepared_reduced)
forest_regressor_mse = mean_squared_error(prices[train_set_index], forest_regressor_prices_predictions)
forest_regressor_rmse = np.sqrt(forest_regressor_mse)
forest_regressor_rmse.round()

In [ ]:
forest_regressor_scores = cross_val_score(forest_regressor, df_prepared_reduced, prices[train_set_index],
                                          scoring="neg_mean_squared_error", cv=K)
forest_regressor_rmse_scores = np.sqrt(-forest_regressor_scores)
display_scores(forest_regressor_rmse_scores)

In [ ]:
assert False

In [ ]:
##################################################################

In [ ]:
# os.chdir(folder_path)
# file_names : list = [i for i in glob.glob("*.{}".format('csv'))]
# df = pd.concat(map(read_car_data, file_names))
